In [74]:
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras


In [75]:
model = "MLP"
#labels = 0
#labels = 1
labels = 2
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [76]:
csv_path = 'jena_climate_2009_2016.csv'
df = pd.read_csv(csv_path)

In [77]:
column_indices = [2, 5]
#column_indices = 5
columns = df.columns[column_indices]
data = df[columns].values.astype(np.float32)

In [78]:
n = len(data)
train_data = data[0:int(n*0.7)]
val_data = data[int(n*0.7):int(n*0.9)]
test_data = data[int(n*0.9):]

In [79]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

In [80]:
input_width = 6
LABEL_OPTIONS = labels

In [81]:
class WindowGenerator:
    def __init__(self, input_width, label_options, mean, std):
        self.input_width = input_width
        self.label_options = label_options
        self.mean = tf.reshape(tf.convert_to_tensor(mean), [1, 1, 2])
        self.std = tf.reshape(tf.convert_to_tensor(std), [1, 1, 2])

    def split_window(self, features):
        inputs = features[:, :-1, :]

        if self.label_options < 2:
            labels = features[:, -1, self.label_options]
            labels = tf.expand_dims(labels, -1)
            num_labels = 1
        else:
            labels = features[:, -1, :]
            num_labels = 2

        inputs.set_shape([None, self.input_width, 2])
        labels.set_shape([None, num_labels])

        return inputs, labels

    def normalize(self, features):
        features = (features - self.mean) / (self.std + 1.e-6)

        return features

    def preprocess(self, features):
        inputs, labels = self.split_window(features)
        inputs = self.normalize(inputs)
        return inputs, labels

    def make_dataset(self, data, train):
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
                data=data,
                targets=None,
                sequence_length=input_width+1,
                sequence_stride=1,
                batch_size=32)
        ds = ds.map(self.preprocess)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds

In [82]:
class MultipleOutputforMAE(tf.keras.metrics.Metric):
  def __init__(self, name='MAE', **kwargs):
      super().__init__(name=name, **kwargs)
      self.total = self.add_weight(name='total', initializer='zeros',shape=(2,))
      self.count = self.add_weight('count',initializer = 'zeros')
  def update_state(self, y_true, y_pred, sample_weight=None):
    error = tf.abs(y_pred - y_true)
    error = tf.reduce_mean(error,axis = 0)
    self.total.assign_add(error)
    self.count.assign_add(1.)

  def reset_states(self):
    self.count.assign(tf.zeros_like(self.count))
    self.total.assign(tf.zeros_like(self.total))
    return

  def result(self):
    result = tf.math.divide_no_nan(self.total,self.count)
    return result

In [83]:
generator = WindowGenerator(input_width, LABEL_OPTIONS, mean, std)
train_ds = generator.make_dataset(train_data, True)
val_ds = generator.make_dataset(val_data, False)
test_ds = generator.make_dataset(test_data, False)

In [100]:
MLPmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2)
])

In [101]:
if labels != 2:
    metrics = [tf.metrics.MeanAbsoluteError()]
else:
    metrics = [MultipleOutputforMAE()]

In [102]:
CNNmodel = keras.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=3,activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

In [103]:
LSTMmodel = keras.Sequential([
    keras.layers.LSTM(units=64),
    keras.layers.Flatten(),
    keras.layers.Dense(1)
])

In [104]:
MLPmodel.compile(optimizer='adam',
              loss=tf.keras.losses.MAE,
              metrics=metrics)

In [105]:
CNNmodel.compile(optimizer='adam',
              loss=tf.keras.losses.MAE,
              metrics=metrics)

In [106]:
LSTMmodel.compile(optimizer='adam',
              loss=tf.keras.losses.MAE,
              metrics=metrics)

In [107]:
MLPmodel.fit(train_ds, epochs=1)
#model.fit(train_ds, batch_size=32, epochs=20)

9200/9200 [==============================] - 14s 1ms/step - loss: 1.9596 - MAE: 1.9595


In [92]:
#CNNmodel.fit(train_ds, batch_size=32, epochs=1)
#CNNmodel.fit(train_ds, batch_size=32, epochs=20)

In [93]:
#LSTMmodel.fit(train_ds, batch_size=32, epochs=1)
#LSTMmodel.fit(train_ds, batch_size=32, epochs=20)

In [108]:
test_loss, test_acc = MLPmodel.evaluate(test_ds, verbose=0)
print('\n MSE for MLP :', test_acc)

#0.22284989058971405
#1.5559452772140503
#1370.72119140625

C:\Users\imanp\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1841: UserWarning: Metric MultipleOutputforMAE implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()



 MSE for MLP : [0.5947353 1.182977 ]


In [95]:
test_loss, test_acc = CNNmodel.evaluate(test_ds, verbose=0)
print('\n MSE for CNN :', test_acc)
#0.08092997968196869
#2.0211997032165527
#1455.0245361328125


 MSE for CNN : [14.08772  60.566593]


In [96]:
test_loss, test_acc = LSTMmodel.evaluate(test_ds, verbose=0)
print('\n MSE for LSTM :', test_acc)
#0.2179471105337143
#2.320437431335449
#1272.8814697265625


 MSE for LSTM : [13.41685 65.19308]


In [109]:
MLPmodel.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_12 (Flatten)        (None, 12)                0         
                                                                 
 dense_24 (Dense)            (None, 128)               1664      
                                                                 
 dense_25 (Dense)            (None, 128)               16512     
                                                                 
 dense_26 (Dense)            (None, 2)                 258       
                                                                 
Total params: 18,434
Trainable params: 18,434
Non-trainable params: 0
_________________________________________________________________


In [110]:
run_model = tf.function(lambda x: MLPmodel(x))
concrete_func = run_model.get_concrete_function(tf.TensorSpec([1, 6, 2],tf.float32))
MLPmodel.save('MLP', signatures=concrete_func)

INFO:tensorflow:Assets written to: MLP\assets


In [111]:
train_ds

<ShuffleDataset shapes: ((None, 6, 2), (None, 2)), types: (tf.float32, tf.float32)>

In [112]:
tf.data.experimental.save(train_ds, './th_train')
tf.data.experimental.save(val_ds, './th_val')
tf.data.experimental.save(test_ds, './th_test')